Load expressions
----------------

imports

In [35]:
from yspecies import *
from typing import List
from yspecies.enums import *
import yspecies.dataset
from yspecies.dataset import *

In [36]:
import pandas as pd
import shap
from pprint import pprint
import random
import numpy as np
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from scipy.stats import kendalltau
from sklearn.utils import resample
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, accuracy_score, recall_score, precision_score, f1_score
#from pyensembl import EnsemblRelease

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# save label encoder to global scope
le = LabelEncoder()
NUMBER_OF_BOOTSTRAPS = 5

## Parameters cell ##

Parameters are overiddent by papermill when run inside DVC stages



In [38]:
# Parameters

animal_class = AnimalClass.mammals.name


### Load pathes ###

In [39]:
from pathlib import Path
if(Path("./data").exists()):
    base_dir = Path("./")
else:
    base_dir = Path("../")
print(base_dir.absolute())

data_dir = base_dir / "data"
input_dir = data_dir / "input"
interim_dir = data_dir / "interim"
output_dir =  data_dir / "output"


/data/sources/species/notebooks/..


In [40]:
expressions_dir = input_dir / "expressions"
by_animal_class_dir = expressions_dir / "by_animal_class"

In [41]:
def load_table(path: Path, index: str = None, dtype: str = None):    
    if index is None:
        return pd.read_csv(str(path), sep="\t", dtype=dtype)
    else:
        return pd.read_csv(str(path), sep="\t", index_col=index, dtype=dtype)

## Load data ##

In [42]:
genes = load_table(interim_dir / "selected_genes.tsv", index="Homo_sapiens")  
samples =  load_table(interim_dir / "selected_samples.tsv" , index="run") 
species =  load_table(interim_dir / "selected_species.tsv" , index="species")
expressions = load_table(by_animal_class_dir / ( animal_class+ ".tsv"), index="reference_gene")
tab(
[
    ["genes", "samples", "species", "expressions"],
    [genes.shape, samples.shape, species.shape, expressions.shape]
]
)

genes,samples,species,expressions
"(14364, 35)","(572, 36)","(47, 16)","(67996, 572)"


In [43]:
expressions.head(3)

,SRR1521445,SRR306838,SRR306839,SRR306840,SRR306841,SRR306842,SRR306843,SRR306844,SRR306845,SRR306847,SRR3715877,SRR5008362,SRR5120939,SRR5120940,SRR5885325,SRR5961830,SRR5961875,SRR5961972,SRR787277,SRR8702484,SRR8991100,SRR6307195,SRR6307196,SRR6307197,SRR6307204,SRR306800,SRR306801,SRR306802,SRR306803,SRR306804,SRR306805,SRR306806,SRR306807,SRR306808,SRR306809,SRR306810,SRR649365,DRR031591,SRR032135,SRR1510173,SRR1758916,SRR1758921,SRR1758922,SRR2040586,SRR306791,SRR306793,SRR095666,SRR3403827,SRR3403828,SRR4039470,SRR4039471,SRR4039473,SRR636850,SRR636900,SRR636945,SRR306826,SRR306827,SRR306828,SRR306829,SRR306831,SRR306832,SRR306833,SRR306834,SRR306835,SRR306836,SRR8750397,SRR8750398,SRR8750399,SRR3195085,SRR3195086,SRR3195096,SRR3195103,SRR3195112,SRR3195113,SRR5080320,SRR5080324,SRR5080392,SRR5080564,SRR1047652,SRR223518,SRR223519,SRR223520,SRR223521,SRR223522,SRR299126,SRR3197665,SRR5990543,SRR6007051,SRR6073386,SRR6073415,SRR1758941,SRR1758942,SRR1758943,SRR1758954,SRR1758957,SRR223512,SRR223513,SRR223514,SRR223515,SRR223516,SRR223517,SRR1759005,SRR1759006,SRR1759007,SRR1759011,SRR1759012,SRR1287653,SRR1287654,SRR1287655,SRR2308103,SRR636887,SRR636888,SRR636932,SRR636933,SRR636977,SRR636978,ERR1331676,SRR2124226,SRR306394,SRR306395,SRR306396,SRR306397,SRR306398,SRR306399,SRR306400,SRR306401,SRR306402,SRR306403,SRR306404,SRR306406,SRR1048140,SRR1048142,SRR2754983,SRR2757329,ERR1331678,ERR1331679,ERR2716205,ERR2716206,ERR2716207,ERR2716208,ERR2716211,ERR2716213,ERR2716214,SRR1334837,SRR3200449,SRR3200452,SRR3200453,SRR3200456,SRR3200460,SRR3200461,SRR3200462,SRR3200463,SRR3200471,SRR3200472,SRR3200473,SRR3218718,SRR636854,SRR636855,SRR636856,SRR636904,SRR636905,SRR636906,SRR636948,SRR636949,SRR636950,SRR3160008,SRR3160018,SRR3160041,SRR3160052,SRR3160061,SRR3194634,SRR3194682,SRR3194698,SRR3194730,SRR3194765,SRR3194791,SRR653846,SRR9720682,SRR1747117,SRR1747118,SRR1747119,SRR1747120,SRR1747121,SRR361433,SRR361434,SRR361435,SRR361436,SRR5118366,SRR5190423,SRR5190425,SRR5190427,SRR5190428,SRR5190430,SRR5190431,SRR5190489,SRR5190491,SRR5190493,SRR5190507,SRR5190508,SRR5190509,SRR636842,SRR636843,SRR636892,SRR636893,SRR636937,SRR636938,SRR1300759,SRR1300763,SRR1300765,SRR1300766,SRR1300767,SRR1300768,ERR2074874,ERR2075072,ERR2075082,ERR2075792,ERR2075986,ERR489282,ERR489283,SRR1013904,SRR1758977,SRR1758978,SRR1758979,SRR1758981,SRR1758982,SRR1758983,SRR1758984,SRR1758985,SRR3109717,SRR3109718,SRR5190444,SRR5190446,SRR5928355,SRR5928357,SRR5928359,SRR765910,SRR866213,SRR306724,SRR306726,SRR306728,SRR306729,SRR306730,SRR306731,SRR306732,SRR306733,SRR306735,SRR306737,SRR5412224,SRR5412225,SRR5412226,SRR5412227,SRR5412228,SRR5412229,SRR5412230,SRR5412231,SRR5412232,SRR5412233,SRR5412234,SRR5412235,SRR5412236,SRR553592,SRR553593,SRR553594,SRR553595,SRR649381,SRR6206908,SRR6206918,SRR1205138,SRR1205218,SRR1205222,SRR1205223,SRR1205998,SRR8708135,SRR1822406,SRR3109726,SRR3109728,SRR489494,SRR5190441,SRR5190453,SRR9024741,SRR9024746,SRR9024747,SRR9024749,SRR9024753,SRR9024755,ERR1331716,SRR3109705,SRR3109709,SRR3109710,SRR3419167,SRR4444968,SRR4444969,SRR4444970,SRR4444971,SRR636839,SRR636840,SRR636934,SRR636935,SRR924544,ERR1331694,SRR1758992,SRR1758995,SRR1758996,SRR1758997,SRR1758998,SRR4444954,SRR4444955,SRR4444956,SRR4444957,SRR4444958,SRR4444959,SRR4444960,SRR4444961,SRR4249993,ERR1331704,ERR1331711,SRR5420457,ERR1331683,ERR1331712,ERR1331715,ERR162228,ERR162229,ERR162240,ERR162246,ERR162251,ERR162267,ERR266373,ERR266376,ERR266380,ERR266381,ERR266394,SRR1200908,SRR1200909,SRR1200911,SRR636846,SRR636847,SRR636848,SRR636896,SRR636897,SRR636898,SRR636941,SRR636942,SRR636943,SRR636852,SRR636853,SRR636902,SRR636903,SRR636946,SRR636947,DRR049384,DRR049385,DRR049386,ERR1331667,ERR1331707,ERR1331708,ERR1331709,SRR6261040,SRR6261041,SRR6261042,SRR6261043,ERR1331671,ERR266370,SRR1786019,SRR1789057,SRR1789059,SRR1789326,SRR1789331,SRR6293952,SRR6293953,SRR6293961,SRR6293972,SRR6293973,SRR6293975,SRR636865,SRR636866,SRR636867,SRR636913,S

In [44]:
exp = expressions.T
exp.index.names = ["run"]
exp.shape

(572, 67996)

In [47]:
samples.reset_index()[['run','bioproject', 'series', 'species', 'tissue', 'sample_name',
       'characteristics']].drop_duplicates().shape

(572, 7)

In [50]:
d = Dataset("selected_species", exp, genes, samples)
d

ValueError: Lengths must match to compare